In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model, Sequential
from keras.preprocessing.image import apply_affine_transform
import os
import cv2
from PIL import Image 
from tqdm import tqdm
from random import shuffle
import pickle
from keras import optimizers
from keras import applications
from keras.initializers import glorot_uniform
# from utils import *

In [4]:
from google.colab import drive
drive.mount('/gdrive/')

Mounted at /gdrive/


In [5]:

training_data = pickle.load(open('/gdrive/MyDrive/CSE344-Computer-Vision-Project/Clothing_Detection/ACS_training_data_64', 'rb'))
labels_dict = pickle.load(open('/gdrive/MyDrive/CSE344-Computer-Vision-Project/Clothing_Detection/ACS_training_labels_dict_64', 'rb'))

In [6]:
#training data sanity check
print('# of training samples:',len(training_data),' shape:',training_data[0][0].shape,
      'shape_label:',training_data[0][1].shape)

# of training samples: 71093  shape: (64, 64, 3) shape_label: (15, 1)


In [7]:
train=training_data[:-3000]
val_test=training_data[-3000:]

In [8]:
X=np.array([np.array(i[0]) for i in train])
Y=np.array([i[1].reshape((len(labels_dict),)) for i in train])

val_test_x=np.array([np.array(i[0]) for i in val_test])
val_test_y=np.array([i[1].reshape((len(labels_dict),)) for i in val_test])

In [9]:
X.shape, Y.shape, val_test_x.shape, val_test_y.shape

((68093, 64, 64, 3), (68093, 15), (3000, 64, 64, 3), (3000, 15))

In [10]:
def CNN_Model(input_shape = (128,128,3), num_classes = 15):
    
    
    X_input = Input(input_shape)
    
    #Stage 1 
    X = Conv2D(64, (5, 5), strides = (1, 1), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X_input)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    
    #Stage 2
    X = Conv2D(64, (5, 5), strides = (1, 1), name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    X = Dropout(0.2)(X)
    
    #Stage 3
    X = Conv2D(128, (3, 3), strides = (1, 1), name = 'conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    X = Dropout(0.2)(X)
    
    #Stage 4 
    X = Conv2D(128, (3, 3), strides = (1, 1), name = 'conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv4')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    X = Dropout(0.2)(X)
    
    #Stage 5
    X = Conv2D(256, (1, 1), strides = (1, 1), name = 'conv5')(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv5')(X)
    X = Activation('relu')(X)
    
    #Stage 6
    X = Flatten()(X)
    X = Dense(1024, activation = 'relu', name = 'fc1')(X)
    X = Dropout(0.2)(X)
    X = Dense(512, activation = 'relu', name = 'fc2')(X)
    X = Dense(num_classes, activation='softmax', name='fc3')(X)
    
    
    model = Model(inputs = X_input, outputs = X, name='CNN_Model')

    return model
    

In [14]:
adadelta = optimizers.Adadelta(learning_rate=0.01, rho=0.95, epsilon=1e-07)
model = CNN_Model(input_shape = (64, 64, 3), num_classes = 15)
model.compile(loss = "categorical_crossentropy", optimizer = adadelta, metrics=["accuracy"])


In [15]:
model.summary()

Model: "CNN_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 60, 60, 64)        4864      
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 60, 60, 64)        256       
_________________________________________________________________
activation_5 (Activation)    (None, 60, 60, 64)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 26, 26, 64)        102464    
_________________________________________________________________
bn_conv2 (BatchNormalization (None, 26, 26, 64)        25

In [16]:
history=model.fit(x=X,y=Y,epochs=40,validation_data=(val_test_x,val_test_y),batch_size=256)

Epoch 1/40
266/266 [==============================] - 35s 127ms/step - loss: 2.5744 - accuracy: 0.1362 - val_loss: 2.4233 - val_accuracy: 0.1930
Epoch 2/40
266/266 [==============================] - 34s 126ms/step - loss: 2.4122 - accuracy: 0.1859 - val_loss: 2.3775 - val_accuracy: 0.1953
Epoch 3/40
266/266 [==============================] - 33s 126ms/step - loss: 2.3782 - accuracy: 0.1979 - val_loss: 2.3520 - val_accuracy: 0.2017
Epoch 4/40
266/266 [==============================] - 34s 126ms/step - loss: 2.3504 - accuracy: 0.2083 - val_loss: 2.3417 - val_accuracy: 0.1977
Epoch 5/40
266/266 [==============================] - 34s 126ms/step - loss: 2.3295 - accuracy: 0.2141 - val_loss: 2.3218 - val_accuracy: 0.2017
Epoch 6/40
266/266 [==============================] - 34s 127ms/step - loss: 2.3100 - accuracy: 0.2220 - val_loss: 2.3126 - val_accuracy: 0.2057
Epoch 7/40
266/266 [==============================] - 34s 126ms/step - loss: 2.3013 - accuracy: 0.2229 - val_loss: 2.3064 - val_ac

In [17]:
model.save('/gdrive/MyDrive/CSE344-Computer-Vision-Project/Clothing_Detection/CNN_Model_40_epoch_29.48_26.73_lr0.01')


INFO:tensorflow:Assets written to: /gdrive/MyDrive/CSE344-Computer-Vision-Project/Clothing_Detection/CNN_Model_40_epoch_29.48_26.73/assets


In [18]:
adadelta = optimizers.Adadelta(learning_rate=0.03, rho=0.95, epsilon=1e-07)
model = CNN_Model(input_shape = (64, 64, 3), num_classes = 15)
model.compile(loss = "categorical_crossentropy", optimizer = adadelta, metrics=["accuracy"])
model.summary()
history=model.fit(x=X,y=Y,epochs=40,validation_data=(val_test_x,val_test_y),batch_size=256)

Model: "CNN_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 60, 60, 64)        4864      
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 60, 60, 64)        256       
_________________________________________________________________
activation_10 (Activation)   (None, 60, 60, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 26, 26, 64)        102464    
_________________________________________________________________
bn_conv2 (BatchNormalization (None, 26, 26, 64)        25

In [19]:
model.save('/gdrive/MyDrive/CSE344-Computer-Vision-Project/Clothing_Detection/CNN_Model_40_epoch_34.21_33.3_lr0.03')


INFO:tensorflow:Assets written to: /gdrive/MyDrive/CSE344-Computer-Vision-Project/Clothing_Detection/CNN_Model_40_epoch_34.21_33.3_lr0.03/assets
